#### calculate cloud base height and identify rainy columns for the entire dataset of MERIAN data
#### Claudia Acquistapace
#### date: 29/11/21
edited on 14/12/2022 to debug cloud base cloud top detection and set basis for virga detection using flags.



In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import matplotlib
import numpy as np
from datetime import datetime,timedelta
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as ticker

def f_closest(array,value):
    '''
    # closest function
    #---------------------------------------------------------------------------------
    # date :  16.10.2017
    # author: Claudia Acquistapace
    # goal: return the index of the element of the input array that in closest to the value provided to the function
    '''
    import numpy as np
    idx = (np.abs(array-value)).argmin()
    return idx  

def f_calc_ct_and_rain_flags(cb, height, height_Ze, flag_cb_source):
    
    '''function to calculate cloud top, cloud thickness and flags for rain based on the inputs:
    - cb: cloud base height found for the given time stamps
    - height: height array coordinate of radarobs
    - height_Ze: heights where Ze is non nan
    - flag_cb_source: flag indicating the source of the cloud base data. It is needed because when there
    is no cloud base from the lidar, if we rely on cloud radar, in rainy conditions cloud base is found 
    at the groud and causes an error. in that case, we take as cloud base the height of the lcl and we consider rain
    all the pixels below.
    output:
    - ct : cloud top identified as the maximum height where there is a Ze non nan
    - thickness: difference between cloud top and cb
    - flag_rain: flag == 1 if there is rain in the column
    - flag_rain_ground : flag ==1 if there is rain below 300 m.'''
    
    # calculating range height around cloud base and height value
    ind_height_closest = f_closest(height, cb)
    height_closest_cb = height[ind_height_closest]
    
    # calculating number of range gates below cloud base and total distance from the ground
    cb_radar = height[0] + np.sum(np.ediff1d(height)[0:ind_height_closest]) # radar height corresponding to cb
    delta_H = np.ediff1d(height)[ind_height_closest] # range gate resolution
    #print('radar range resolution at cloud base: ', cb_radar, delta_H)
    
    # calculate cloud top and cloud geometrical thickness
    ct = np.nanmax(height_Ze)
    thickness = ct - cb

    # calculate rain flag if cb is from doppler lidar or from radar with no rain
    if flag_cb_source != 2:
        
        # calculate number of range gates between cloud base and the surface
        N_range_surf = len(np.ediff1d(height)[0:ind_height_closest])
        n_ranges_ze_below_cb = int((height_closest_cb - np.nanmin(height_Ze))/delta_H)

        if (n_ranges_ze_below_cb > 0) and (n_ranges_ze_below_cb/N_range_surf > 0.1):
            flag_rain = 1 # almost all column between cb and ze min has signal, so it is a rainy column
        else:
            flag_rain = 0 # no rain in the column
    else:
        flag_rain = 1 # assigning rain flag if cloud base is assigned with lcl because 
                            # in that case no cb from doppler is found and Ze has long vertical profile

    # check for rain at the surface
    if (np.nanmin(height_Ze) < 300.): 
        flag_rain_ground = 1 # rain at the ground
    else:
        flag_rain_ground = 0 # 

    return(ct, thickness, flag_rain, flag_rain_ground)

def f_save_to_ncdf(time, flag_rain, flag_rain_ground, ct, cb, thickness, virga_depth, flag_cb_source, date, path_out):
    '''function to save all variables in a ncdf file'''  
    # save xarray dataset containing the correction terms for the hour
    dims             = ['time']
    coords           = {"time":time}

    flag_cloud_base_source         = xr.DataArray(dims=dims, coords=coords, data=flag_cb_source,
                             attrs={'long_name':'flag for cloud base source',
                                    'units':'', 
                                   'comment':'= 0 : cloud base from doppler lidar, \
                                              = 1 : cloud base from cloud radar'})

    flag_rain_DA                   = xr.DataArray(dims=dims, coords=coords, data=flag_rain,
                             attrs={'long_name':'flag for rain below cloud base',
                                    'units':'', 
                                   'comment':'= 1 : rain is present in the vertical column, \
                                              = 0 : no rain in the column '})

    flag_rain_ground_DA            = xr.DataArray(dims=dims, coords=coords, data=flag_rain_ground,
                             attrs={'long_name':'flag rain at the ground',
                                    'units':'', 
                                   'comment':'= 1 : rain at the surface, \
                                              = 0 : no rain at the surface'})
    
    virga_depth_DA            = xr.DataArray(dims=dims, coords=coords, data=virga_depth,
                             attrs={'long_name':'depth of virga layer',
                                    'units':'m', 
                                   'comment':''})
    
    cth =  xr.DataArray(dims=dims, coords=coords, data=ct,
                             attrs={'long_name':'cloud top height',
                                    'units':'m', 
                                   })
    cloud_thickness =  xr.DataArray(dims=dims, coords=coords, data=thickness,
                             attrs={'long_name':'cloud geometrical thickness',
                                    'units':'m', 
                                   })
    cloud_base = xr.DataArray(dims=dims, coords=coords, data=cb,
                             attrs={'long_name':'cloud base height',
                                    'units':'m', 
                                   'comment':''})

    variables         = {'flag_cloud_base_source':flag_cloud_base_source, 
                         'flag_rain': flag_rain_DA, 
                         'flag_rain_ground':flag_rain_ground_DA, 
                         'virga_depth':virga_depth_DA,
                         'cloud_top_height':cth,
                         'cloud_geometrical_thickness':cloud_thickness,
                         'cloud_base':cloud_base}
    global_attributes = {'CREATED_BY'       : 'Claudia Acquistapace',
                        'CREATED_ON'       :  str(datetime.now()),
                        'FILL_VALUE'       :  'NaN', 
                        'PI_NAME'          : 'Claudia Acquistapace',
                        'PI_AFFILIATION'   : 'University of Cologne (UNI), Germany', 
                        'PI_ADDRESS'       : 'Institute for geophysics and meteorology, Pohligstrasse 3, 50969 Koeln', 
                        'PI_MAIL'          : 'cacquist@meteo.uni-koeln.de',
                        'DATA_DESCRIPTION' : 'flags and geometrical cloud properties for MSM data obtained from sinergy of wband radar, Doppler lidar and ship-based obs',
                        'DATA_DISCIPLINE'  : 'Atmospheric Physics - Remote Sensing Profilers',
                        'DATA_GROUP'       : 'Experimental;Profile;Moving',
                        'DATA_SOURCE'      : '',
                        'DATA_PROCESSING'  : 'https://github.com/ClauClouds/rain_obs_paper/',
                        'INSTRUMENT_MODEL' : 'wband, Doppler lidar, in_situ ship based',
                         'COMMENT'         : '' }
    dataset    = xr.Dataset(data_vars = variables,
                                      coords = coords,
                                       attrs = global_attributes)
    dataset.to_netcdf(path_out+date+'_flags_cloud_properties.nc')       
    return()


def f_plot_quicklook_flags_cb_ct(date, data, ct, thickness, cb_arr, flag_cb, flag_rain, flag_rain_ground, path_out_plots):


    datetime_day = pd.to_datetime(data['time'].values)
    height = data['height'].values

    fig, axs = plt.subplots(5, 1, figsize=(16,16), sharex=True, constrained_layout=True)

    mesh = axs[0].pcolormesh(datetime_day, height, data.radar_reflectivity.values.T, vmin=-50.,                              vmax=20., cmap='jet', rasterized=True)
    #axs[0].plot(datetime_day, data['lcl'].values, label='lcl')
    axs[0].plot(datetime_day, cb_arr, color='red', label='cb')
    cbar = fig.colorbar(mesh, ax=axs[0], label='reflectivity',location='right', aspect=20, use_gridspec=True)
    axs[0].set_ylim(500., 2500.)
    axs[0].legend(frameon=False)
    axs[0].set_ylabel('Height [m]')

    axs[1].scatter(datetime_day, flag_rain, color='blue')
    axs[1].set_ylim(0., 1.5)
    axs[1].set_ylabel('flag_rain')

    axs[2].plot(datetime_day, flag_rain_ground, color='blue')
    axs[2].set_ylim(0., 1.5)
    axs[2].set_ylabel('flag_rain_ground')

    axs[3].plot(datetime_day, flag_cb, color='blue')
    axs[3].set_ylim(0., 3.5)
    axs[3].set_ylabel('flag_cb')

    axs[4].plot(datetime_day, thickness, color='blue')
    axs[4].set_ylim(0., 1000.)
    axs[4].set_ylabel('cloud thickness [m]')

    for ax, l in zip(axs[:].flatten(), ['(a)',  '(b)',  '(c)',  '(d)',  '(e)']):
        ax.text(-0.05, 1.1, l,  fontweight='black', fontsize=16, transform=ax.transAxes)
        #ax.set_xlim(SST_binned_arr[0]-0.1, SST_binned_arr[-1]+0.1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["bottom"].set_linewidth(3)
        ax.spines["left"].set_linewidth(3)
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(n=5))
        ax.tick_params(which='minor', length=5, width=2)
        ax.tick_params(which='major', length=7, width=3)
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(n=5))
        ax.tick_params(axis='both', labelsize=32)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        ax.set_xlim(datetime_day[0], datetime_day[-1])
        ax.set_xlabel('Time UTC [dd/mm hh]')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))#('%d/%m %H:%M'))

    figure_name = path_out_plots+date+'_fig_quicklook_flags.png'
    plt.savefig(figure_name)
    #plt.imsave(fig, figure_name)
    return()


# path to Wband radar data
path_w_band = '/data/obs/campaigns/eurec4a/msm/wband_radar/ncdf/published_data_v2/'
path_out = '/data/obs/campaigns/eurec4a/msm/flags/new_version/'
path_cb_lidar = '/data/obs/campaigns/eurec4a/msm/flags/'

#### reading and merging data from wband, lcl, cloud base derived from doppler lidar

In [2]:
# reading one daily file from Wband radar and checking signal below cloud base. 
# Also check if for every cloud base there is a radar bin, and quantify missed cloud bases from Wband radar
# set up a rain flag based on reflectivity column below cloud base

Eurec4aDays     = pd.date_range(datetime(2020,1,19),datetime(2020,2,19),freq='d')

# loop on days: read the data, calculate cloud base height using max gradient algorithm, and store ncdf
for iDay, day in enumerate(Eurec4aDays):
    
    # reading day to be processed
    date = str(day)[0:4]+str(day)[5:7]+str(day)[8:10]
    print('processing day: '+date)

    if os.path.isfile(path_out+date+'_flags_cloud_properties.nc'):
        print(date+' - already processed')
    else:
    
        # reading wband radar data of the selected day
        w_band_data_day = xr.open_dataset(path_w_band+date+'_wband_radar_msm_eurec4a_intake.nc')
        
        # removing double time indeces when present
        _, index = np.unique(w_band_data_day['time'], return_index=True)
        w_band_data_day = w_band_data_day.isel(time=index)
        #w_band_data_day = w_band_data_day.loc[~w_band_data_day.index.duplicated(keep='first')]
        
        # reading lcl height array
        lcl_data = xr.open_dataset('/data/obs/campaigns/eurec4a/msm/LCL_dataset.nc')
        lcl_data_day = lcl_data.sel(time=slice(datetime(int(date[0:4]), int(date[4:6]),int(date[6:8]),0,0,0),\
                                              datetime(int(date[0:4]), int(date[4:6]),int(date[6:8]),23,59,59)))
        # dropping variables that are double
        lcl_data_day = lcl_data_day.drop('lat')
        lcl_data_day = lcl_data_day.drop('lon')


        # interpolating lcl data on wband radar data
        lcl_data_interp = lcl_data_day.interp(time=w_band_data_day.time.values)

        if (date != '20200119') * (date != '20200219'): 
            
            # reading doppler lidar data of the selected day
            cb_data_day = xr.open_dataset(path_cb_lidar+date+'_cb_Doppler_lidar.nc')
            # removing double index of time in lidar data
            _, index2 = np.unique(cb_data_day['time'], return_index=True)
            cb_data_day = cb_data_day.isel(time=index2)        

            # interpolate cb data on radar time and range resolution
            cb_data_interp = cb_data_day.interp(time=w_band_data_day.time.values)
            cb_data_interp = cb_data_interp.interp(height=w_band_data_day.height.values)

            # merging the three datasets together, compat='override')
            data_day = xr.merge([w_band_data_day,cb_data_interp, lcl_data_interp])
        else:
            # merging the three datasets together , compat='override')
            data_day = xr.merge([w_band_data_day, lcl_data_interp])
            
        # reading variables from the merged file to determine flags
        height = data_day.height.values
        time = pd.to_datetime(data_day.time.values)
        Ze = w_band_data_day.radar_reflectivity.values
        Vd = w_band_data_day.mean_doppler_velocity.values       
        
        # initializing arrays to nans
        ct = np.zeros((len(time)))
        thickness = np.zeros((len(time)))
        flag_rain = np.zeros((len(time))) 
        flag_cb_source = np.zeros((len(time)))  
        flag_rain_ground = np.zeros((len(time)))
        cb_arr = np.zeros((len(time)))
        virga_depth = np.zeros((len(time)))
        
        flag_rain.fill(np.nan)
        flag_cb_source.fill(np.nan)
        ct.fill(np.nan)
        thickness.fill(np.nan)
        cb_arr.fill(np.nan)
        flag_rain_ground.fill(np.nan)
        virga_depth.fill(np.nan)
        
        ''' 
        flag values:
        flag_cb_source:
            = nan: no data
            = 0 : cloud base from doppler lidar
            = 1 : cloud base from cloud radar
            = 2 : 
        
        flag_rain:
            = nan: no data
            = 0 : no rain is present in the vertical column
            = 1 : rain in the column 
        flag_rain_ground:
            = nan: no data
            = 0 : no rain at the surface
            = 1 : rain at the surface
        '''

        for ind, time_val in enumerate(time):

            # find the heights at which reflectivity is different from nan
            height_Ze = height[np.isfinite(Ze[ind, :])]
            ZE_values = Ze[ind, np.isfinite(Ze[ind, :])]
            VD_values = Vd[ind, np.isfinite(Ze[ind, :])]
            
            # reading cloud base height from doppler lidar and lcl from ship data
            lcl = data_day.lcl.values[ind]   
            
            # for days where there are no lidar data we set cb = np.nan
            if (date != '20200119') * (date != '20200219'):
                cb = data_day.cb.values[ind]
            else:
                cb = np.nan

            # there are reflectivity values in the column and cloud base exists
            if ((len(height_Ze) != 0) and (np.isfinite(cb))): 
                # establishing reference for cloud base height using wind lidar
                flag_cb_source[ind] = 0 # cloud base from doppler lidar
                #print('cloud base source: doppler lidar')
                cb_arr[ind] = cb
                # calculating closest radar cloud signal to cloud base
                height_closest_cb = height_Ze[f_closest(height_Ze, cb_arr[ind])]
                
                # calculate cloud top, thickness and flags for rain
                ct[ind], thickness[ind], flag_rain[ind], flag_rain_ground[ind] = f_calc_ct_and_rain_flags(cb_arr[ind], height, height_Ze, flag_cb_source[ind])
            
            # there are reflectivity values in the column but cloud base does from lidar not exist
            if ((len(height_Ze) != 0) and (~np.isfinite(cb))):
                
                # find the heights at which reflectivity is different from nan
                height_Ze = height[np.isfinite(w_band_data_day.radar_reflectivity.values[ind, :])]
                
                # find minimum height where Ze is not nan
                Ze_lowest_height = np.nanmin(height_Ze)
                
                # compare lowest Ze height with lcl: if it is within 200 m from lcl then is equal to cb, 
                # otherwise cb is given by lcl and we are in presence of rain
                diff_height = Ze_lowest_height - lcl
                if diff_height > 0.:
                    flag_cb_source[ind] = 1 # cloud base from cloud radar
                    #print('cloud base source: cloud radar')
                    cb_arr[ind] = np.nanmin(height_Ze)
                    height_closest_cb = np.nanmin(height_Ze)
                    ind_cb_found = np.nanargmin(height_Ze)
                else:
                    flag_cb_source[ind] = 2 # cloud base from lcl, rainy or virga conditions
                    #print('cloud base source: cloud radar')
                    cb_arr[ind] = lcl
                    ind_cb_found = f_closest(height_Ze, lcl)
                    
                # check if cloud base found is in rain and iterate to find a new cloud base eventually
                if VD_values[ind_cb_found] < -0.4:

                    #find first value of VD above the cloud base that has VD >-0.4
                    ind_cloud_above_rain = np.where(VD_values[ind_cb_found:-1] > -0.4)[0]
                    if len(ind_cloud_above_rain) > 0:
                        ind_cb_found_new = np.nanmin(ind_cloud_above_rain)
                        # readapting cb to the new found value
                        cb_arr[ind] = height_Ze[ind_cb_found_new]
                    else:
                        cb_arr[ind] = np.nan
                # calculate cloud top, thickness and flags for rain
                #print('before')
                ct[ind], thickness[ind], flag_rain[ind], flag_rain_ground[ind] = f_calc_ct_and_rain_flags(cb_arr[ind], height, height_Ze, flag_cb_source[ind])
                #print('after')
                
            
            # calculate virga depth in case of virga conditions
            if (flag_rain[ind] == 1) * (flag_rain_ground[ind] == 0):
                virga_depth[ind] = cb_arr[ind] - np.nanmin(height_Ze)
                #print(virga_depth[ind])
 
        
        #producing quicklooks
        #   
        #print('saving file')
        saved = f_save_to_ncdf(time, flag_rain, flag_rain_ground, ct, cb_arr, thickness, virga_depth, flag_cb_source, date, path_out)        
        print('saved file')



processing day: 20200119
20200119 - already processed
processing day: 20200120
20200120 - already processed
processing day: 20200121
20200121 - already processed
processing day: 20200122
20200122 - already processed
processing day: 20200123
20200123 - already processed
processing day: 20200124
20200124 - already processed
processing day: 20200125
20200125 - already processed
processing day: 20200126
20200126 - already processed
processing day: 20200127
20200127 - already processed
processing day: 20200128
20200128 - already processed
processing day: 20200129
20200129 - already processed
processing day: 20200130
20200130 - already processed
processing day: 20200131
20200131 - already processed
processing day: 20200201
20200201 - already processed
processing day: 20200202
20200202 - already processed
processing day: 20200203
20200203 - already processed
processing day: 20200204
20200204 - already processed
processing day: 20200205
20200205 - already processed
processing day: 20200206
202

In [ ]:
# producting quicklooks
for iDay, day in enumerate(Eurec4aDays):
    
    # reading day to be processed
    date = str(day)[0:4]+str(day)[5:7]+str(day)[8:10]
    print('processing day: '+date)

    if os.path.isfile(path_out+date+'_fig_quicklook_flags.png'):
        print(date+' - already processed')
    else:
        print('plotting quicklooks')
        #plot_done = f_plot_quicklook_flags_cb_ct(date, data_day, ct, thickness, cb_arr, flag_cb_source, flag_rain, flag_rain_ground, path_out)
        data = xr.open_dataset(path_w_band+date+'_wband_radar_msm_eurec4a_intake.nc')
        datetime_day = pd.to_datetime(data['time'].values)
        flag_data = xr.open_dataset(path_out+date+'_flags_cloud_properties.nc')
        cb_arr = flag_data.cloud_base.values
        flag_rain = flag_data.flag_rain.values
        thickness = flag_data.cloud_geometrical_thickness.values
        flag_cb = flag_data.flag_cloud_base_source.values
        flag_rain_ground = flag_data.flag_rain_ground.values
        height = data['height'].values

        fig, axs = plt.subplots(5, 1, figsize=(8,8), sharex=True, constrained_layout=True)

        mesh = axs[0].pcolormesh(datetime_day, height, data.radar_reflectivity.values.T, vmin=-50.,\
                                 vmax=20., cmap='viridis', rasterized=True)
        #axs[0].plot(datetime_day, data['lcl'].values, label='lcl')
        axs[0].plot(datetime_day, cb_arr, color='red', label='cb')
        cbar = fig.colorbar(mesh, ax=axs[0], label='reflectivity',location='right', aspect=20, use_gridspec=True)
        axs[0].set_ylim(500., 2500.)
        axs[0].legend(frameon=False)
        axs[0].set_ylabel('Height [m]')

        axs[1].scatter(datetime_day, flag_rain, color='blue')
        axs[1].set_ylim(0., 1.5)
        axs[1].set_ylabel('flag_rain')

        axs[2].plot(datetime_day, flag_rain_ground, color='blue')
        axs[2].set_ylim(0., 1.5)
        axs[2].set_ylabel('flag_rain_ground')

        axs[3].plot(datetime_day, flag_cb, color='blue')
        axs[3].set_ylim(0., 3.5)
        axs[3].set_ylabel('flag_cb')

        axs[4].plot(datetime_day, thickness, color='blue')
        axs[4].set_ylim(0., 1000.)
        axs[4].set_ylabel('cloud thickness [m]')

        for ax, l in zip(axs[:].flatten(), ['(a)',  '(b)',  '(c)',  '(d)',  '(e)']):
            ax.text(-0.05, 1.1, l,  fontweight='black', fontsize=16, transform=ax.transAxes)
            #ax.set_xlim(SST_binned_arr[0]-0.1, SST_binned_arr[-1]+0.1)
            ax.spines["top"].set_visible(False)
            ax.spines["right"].set_visible(False)
            ax.spines["bottom"].set_linewidth(3)
            ax.spines["left"].set_linewidth(3)
            ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(n=5))
            ax.tick_params(which='minor', length=5, width=2)
            ax.tick_params(which='major', length=7, width=3)
            ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(n=5))
            ax.tick_params(axis='both', labelsize=32)
            ax.get_xaxis().tick_bottom()
            ax.get_yaxis().tick_left()
            ax.set_xlim(datetime_day[0], datetime_day[-1])
            ax.set_xlabel('Time UTC [dd/mm hh]')
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))#('%d/%m %H:%M'))

        figure_name = path_out+date+'_fig_quicklook_flags.png'
        fig.savefig(figure_name)

        
    print('plot done for date ' + date)

processing day: 20200119
plotting quicklooks
plot done for date 20200119
processing day: 20200120
plotting quicklooks


/tmp/ipykernel_192137/2984451933.py:69: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.savefig(figure_name)


plot done for date 20200120
processing day: 20200121
plotting quicklooks
plot done for date 20200121
processing day: 20200122
plotting quicklooks
plot done for date 20200122
processing day: 20200123
plotting quicklooks
